# Compare FSE/VarBugs

In [ ]:
#Set Baseline and Experiment
tool = "clang" #clang, infer, phasar
program = "varbugs" #axtls, toybox, busybox, varbugs

import itertools
import os
import random
import json
import string
from json import JSONEncoder

def _default(self, obj):
    return getattr(obj.__class__, "to_json", _default.default)(obj)

_default.default = JSONEncoder().default
JSONEncoder.default = _default

from pathos import multiprocessing
import logging
import copy
from pathlib import Path
from typing import Tuple
import re
from z3.z3 import Solver, And, Or, Not, Bool, Int, sat

baselines: Path = Path(('preparedFSEBaselines/' if program != 'varbugs' else 'preparedVBDbBaselines/') + tool + program + ".json")
experimental_results: Path = Path('preparedExperimentResults/' + tool + program + '.json')

with open(baselines) as f:
    baselines = json.load(f)

with open(experimental_results) as f:
    experimental_results = json.load(f)

lonely_baselines = copy.deepcopy(baselines)
lonely_experimental_results = copy.deepcopy(experimental_results)

class IntRange:
    def __init__(self, lower_bound_inclusive, upper_bound_exclusive):
        self.lower_bound_inclusive = lower_bound_inclusive
        self.upper_bound_exclusive = upper_bound_exclusive

    def __contains__(self, item):
        return isinstance(item, int) and (self.lower_bound_inclusive <= item < self.upper_bound_exclusive)

    def __repr__(self):
        return f"IntRange({self.lower_bound_inclusive, self.upper_bound_exclusive})"

    def __str__(self):
        return f"[{self.lower_bound_inclusive}:{self.upper_bound_exclusive})"

    def to_json(self):
        return str(self)


for e in experimental_results:
    toks = e['original_line'].split(':')
    try:
        e['original_line'] = IntRange(int(toks[0]), int(toks[1]) + 1)
    except Exception as ex:
        e['original_line'] = []
    #print('\t'.join(["experimental", *[str(s) for s in e.values()]]).replace("\n", ""))

    if e['function_line_range'] == 'ERROR':
        e['function_line_range'] = []
    else:
        toks = e['function_line_range'].split(':')
        try:
            e['function_line_range'] = IntRange(int(toks[1]), int(toks[2]) + 1)
        except Exception as ex:
            logging.exception(f"e was {e}")
    e['presence_condition'] = str(e['presence_condition'])

print(f"We have {len(baselines)} baseline results.")
print(f"We have {len(experimental_results)} experimental results.")

In [0]:
import tqdm


def match_stats(baseline_result: dict, experimental_result: dict) -> Tuple:
    """
    Returns a vector of different match information.
    (a, b, c)
    a = True iff baseline and experimental have the same line number, message, and file.
    b = True iff baseline and experimental have the same message, file, and baseline is within experimental's function scope.
    c = True iff baseline's configuration is compatible with experimental's presence condition.
    """

    a = (baseline_result['sanitized_message'].lstrip().rstrip() == experimental_result['sanitized_message'].lstrip().rstrip() and \
         baseline_result['input_line'] in experimental_result['original_line'] and\
         baseline_result['input_file'].split('.')[0] == experimental_result['input_file'].split('.')[0])

    b = (baseline_result['sanitized_message'].lstrip().rstrip() == experimental_result['sanitized_message'].lstrip().rstrip() and \
         baseline_result['input_line'] in experimental_result['function_line_range'] and\
         baseline_result['input_file'].split('.')[0] == experimental_result['input_file'].split('.')[0])

    c = False

    if experimental_result['feasible'] and 'Or(None' not in experimental_result['presence_condition'] and experimental_result['presence_condition'] not in ['Or(None)', 'None'] and (a or b):  # Don't bother doing this expensive step when the file and line number are different.
        baseline_var_mapping = {}
        for var in baseline_result['configuration']:
            if type(var) is str:
                if var.startswith('DEF'):
                    baseline_var_mapping[re.sub(r"^(DEF_.*)", r"\1", var)] = True
                elif var.startswith('UNDEF'):
                    baseline_var_mapping[re.sub(r"^UN(DEF_.*)", r"\1", var)] = False
                else:
                    raise RuntimeError(f"Don't know how to handle variable {var}")

        s = Solver()
        for var, val in baseline_var_mapping.items():
            var = Bool(var)
            if val:
                s.add(var)
            else:
                s.add(Not(var))

        for mat in re.findall("DEF_[a-zA-Z0-9_]+", experimental_result['presence_condition']):
            exec(f"{mat} = Bool('{mat}')")
           
        for mat in re.findall("USE_[a-zA-Z0-9_]+", experimental_result['presence_condition']):
            exec(f"{mat} = Int('{mat}')")

        while True:
            try:
                s.add(eval(experimental_result['presence_condition']))  # TODO Definitely need to do more transformation here.
                break
            except NameError as ne:
                var = re.search("name '(.*)' is not defined", str(ne))
                exec(f"{var.group(1)} = Int('{var.group(1)}')")
        c = s.check() == sat
    return a, b, c

def tupleize(func, args): return func(*args), tuple(args)

summary = {}

# Note that results depend on the order of keys in this dictionary, because once we find a match_stats for one level we do not keep searching for the next.
#  E.g., for a given report, we will first look for results with which it has a (True, True, True) report. If it has one, we do not continue searching for
#  matches for (False, True, True), (True, False, True), etc.
result_hierarchy = {(True, True, True): 0, (False, True, True): 0, (True, False, True): 0, (True, True, False): 0, (False, True, False): 0, (False, False, True): 0, (True, False, False): 0, (False, False, False): 0}

report = []
for b in tqdm.tqdm(baselines):
    # Results are (baseline, desugared, match tuple)
    results = [(b, e, match_stats(b, e)) for e in experimental_results]
    found = False
    for r in result_hierarchy.keys():
        for res in results:
            if res[2] == r:
                found = True
                result_hierarchy[r] += 1
                # -----
                # Here is where you compile information about any specific reports you need. This block of code
                # iterates through all baselines and finds the highest level of matching that is available.
                # So, for example, if you wanted to collect all of the unmatched originals, you would uncomment out this line of code:
                #
                if (r != (True, True, True) and r != (False, True, True)):
                    report.append(res[0])
                break # DO NOT DELETE THE BREAK!
        if found:
            break
print(len(report),'/',len(baselines))
for r in report:
    for key in ['id','input_file','input_line','message','warning_path']:
        print(key,':',r[key])

# Results

In [ ]:
print(f"Number of baseline results: {len(baselines)}")
print(f"Number of desugared results: {len(experimental_results)}")
print(f"Number of feasible desugared results: {len(list(filter(lambda e: e['feasible'],experimental_results)))}")
print(f"Number of exact matches: {result_hierarchy[(True, True, True)]}")
print(f"Number of partial matches: {result_hierarchy[False, True, True]}")
print(f"Number of unmatched: {sum(v for k, v in result_hierarchy.items() if k not in [(True, True, True), (False, True, True)])}")

In [ ]:
results = dict()

for b in baselines:
    conf = str(b.get('configuration'))
    if conf not in results:
        results[conf] = 0
    results[conf] += 1
print(f"Number of configurations with warnings: {len(results)}")

for b in baselines:
    conf = str(b.get('configuration'))
    if results[conf] == 6:
        for key in ['id','input_file','input_line','message','warning_path']:
            print(key,':',b[key])
import random
while len(results.keys()) < 1000:
    results[''.join(random.choices(string.ascii_letters, k=5))] = 0

print(f"Average warnings per config is {float(sum(results.values()))/float(len(results.values()))}")

In [ ]:
result_hierarchy = {(True, True, True): 0, (False, True, True): 0, (True, False, True): 0, (True, True, False): 0, (False, True, False): 0, (False, False, True): 0, (True, False, False): 0, (False, False, False): 0}
finds = 1
report = []
for e in tqdm.tqdm(experimental_results):
    results = [(b, e, match_stats(b, e)) for b in baselines]
    found = False
    for r in result_hierarchy.keys():
        for res in results:
            if res[2] == r:
                found = True
                result_hierarchy[r] += 1
                if (r != (True, True, True) and r != (False, True, True)):
                    if 'verified' in e.keys() and e['verified'] != 'UNVERIFIED':
                        print(finds,e['verified'])
                        finds += 1
                        for key in ['id','input_file','input_line','message','warning_path','configuration']:
                            print(key,':',e[key])
                    else:
                        for key in ['id','input_file','input_line','message','warning_path']:
                            print(key,':',e[key])

                break
        if found:
            break

print('-----------')
print(f"Number of desugared results: {len(experimental_results)}")
print(f"Number of baseline results: {len(baselines)}")
print(f"Number of exact matches: {result_hierarchy[(True, True, True)]}")
print(f"Number of partial matches: {result_hierarchy[False, True, True]}")
print(f"Number of unmatched: {sum(v for k, v in result_hierarchy.items() if k not in [(True, True, True), (False, True, True)])}")

In [ ]:
#STOP
break

# Deduplicate & Map FSE Baseline

In [31]:
#Set Input and Output

tool = "infer" #clang, infer, phasar
program = "toybox" #axtls, toybox, busybox

import copy
import itertools
import os
import random
import json
import string
from json import JSONEncoder
def _default(self, obj):
    return getattr(obj.__class__, "to_json", _default.default)(obj)
_default.default = JSONEncoder().default
JSONEncoder.default = _default
from pathos import multiprocessing
import logging
import copy
from pathlib import Path
from typing import Tuple
import re

inputLocation = f"/home/austin/Downloads/SugarlyzerResults/FSEBaselines/baselines/{tool}/{program}/results.json"
mapLocation = f"/home/austin/Downloads/SugarlyzerResults/FSEMappings/{program}.json"
outputLocation = f"/home/austin/Downloads/SugarlyzerResults/preparedFSEBaselines/{tool}{program}.json"

baselines: Path = Path(inputLocation)
maps: Path = Path(mapLocation)

with open(baselines) as f:
    baselines = json.load(f)
with open(maps) as f:
    maps = json.load(f)

for b in baselines:
    try:
        b['original_configuration'] = [re.search(r"(\d*)\.config", b['input_file']).group(1)]
    except AttributeError as ae:
        b['original_configuration'] = []
    assert(b['original_configuration'] is not None)
    parts= b['input_file'].split('/')
    b['input_file'] = '/' + '/'.join([parts[1],'/'.join(parts[3:])])
    
def eq(a,b):
    for stat in ['input_file','input_line','message']:
        if a[stat] != b[stat]:
            return False
    if len(a['warning_path']) != len(b['warning_path']):
        return False
    for w in a['warning_path']:
        if w not in b['warning_path']:
            return False
    return True
    
deduped = []
for b in baselines:
    assert(b['original_configuration'] is not None)
    found = False
    for d in deduped:
        assert(d['original_configuration'] is not None)
        if eq(b,d):
            found = True
            d['original_configuration'].extend(b['original_configuration'])
            break
    if not found:
        deduped.append(b)

for d in deduped:
    for c in d['configuration']:
        fixxer = c[0]
        if c[1] == False:
            fixxer = "!DEF_" + fixxer
        elif c[1] == "y":
            fixxer = "DEF_" + fixxer
        else:
            fixxer = "USE_" + fixxer + " == " + c[1]
        for k,v in maps.items():
            if fixxer == v:
                temp = k.replace('!DEF','UNDEF')
                d['configuration'].append(temp)
        
with open(outputLocation, 'w') as f:
    json.dump(deduped, f, indent=2)
    

# Sample Configurations

In [36]:
import json
import random

with open("/home/austin/Desktop/boxplot_data.csv", 'w') as outfile:
    outfile.write("tool,program,iteration,sample_size,num_bugs")
    for tool in ['clang', 'infer', 'phasar']:
        for target in ['axtls', 'busybox', 'toybox']:
            input_file = f"/home/austin/Downloads/SugarlyzerResults/preparedFSEBaselines/{tool}{target}.json"
            
            with open(input_file) as f:
                inputs = json.load(f)
                
            for j in [10, 50, 100, 500]:
                for i in range(50):
                    sample = random.sample(range(1000), j)
                    bugs = copy.deepcopy(inputs)
                    num_bugs = 0
                    for s in sample:
                        for a in bugs:
                            if str(s) in a['original_configuration']:
                                num_bugs += 1
                                bugs.remove(a)
                                
                    outfile.write(f"{tool},{target},{i},{j},{num_bugs}\n")
    

# Deduplicate Varbugs Baselines

In [ ]:
#Set Input and Output

tool = "clang" #clang, infer, phasar

import copy
import itertools
import os
import random
import json
import string
from json import JSONEncoder
def _default(self, obj):
    return getattr(obj.__class__, "to_json", _default.default)(obj)
_default.default = JSONEncoder().default
JSONEncoder.default = _default
from pathos import multiprocessing
import logging
import copy
from pathlib import Path
from typing import Tuple
import re
from z3.z3 import Solver, And, Or, Not, Bool, Int, sat

inputLocation = f"VBDbBaselines/{tool}varbugs.json"
outputLocation = f"preparedVBDbBaselines/{tool}varbugs.json"

baselines: Path = Path(inputLocation)

with open(baselines) as f:
    baselines = json.load(f)
    
results = dict()
for e in baselines:
    key = (e['message'], e['input_file'], e['input_line'])
    if key not in results:
        results[key] = []
    results[key].append(e)

def subset(a,b):
    al = a['configuration']
    bl = b['configuration']
    if len(a) == len(b):
        return (False,None)
    smaller = al if len(al) < len(bl) else bl
    bigger = al if len(al) > len(bl) else bl
    allIn = True
    for x in smaller:
        if x not in bigger:
            allIn = False
            break
    if allIn:
        aa = copy.deepcopy(a)
        aa['configuration'] = smaller
        return (True, aa)
    return (False, None)

def common(a,b):
    al = a['configuration']
    bl = b['configuration']
    if len(a) != len(b):
        return (False,None)
    swapIn = []
    for x in al:
        if x not in bl:
            swap = x[2:] if x.startswith('UNDEF') else 'UN' + x
            if swap not in bl:
                return (False, None)
        else:
            swapIn.append(x)
    if len(swapIn) == len(al) -1:
        aa = copy.deepcopy(a)
        aa['configuration'] = swapIn
        return (True, aa)
    return (False, None)

def eq(a,b):
    if len(a) != len(b):
        return False
    for x in a['configuration']:
        if x not in b['configuration']:
            return False
    return True

dedupe = []
for k,v in results.items():
    #v = list of all same bug
    anyMatch = True
    while anyMatch:
        anyMatch = False
        newList = []
        i = 0
        while i < len(v):
            matched = False
            j = i + 1
            while j < len(v):
                res = subset(v[i],v[j])
                if res[0]:
                    newList.append(res[1])
                    matched = True
                    anyMatch = True
                    v.pop(j)
                    continue
                res = common(v[i],v[j])
                if res[0]:
                    newList.append(res[1])
                    matched = True
                    anyMatch = True
                    v.pop(j)
                    continue
                if eq(v[i],v[j]):
                    v.pop(j)
                    continue
                j += 1
            if not matched:
                newList.append(v[i])
            i += 1
        v = newList
    dedupe.extend(newList)
    
with open(outputLocation, 'w') as f:
    json.dump(dedupe, f, indent=2)

# Deduplicate/Clean ExperimentResults

In [ ]:
#Set Input and Output

tool = "clang" #clang, infer, phasar
program = "varbugs" #axtls, toybox, busybox, varbugs, tosemuclibc, tosembusybox, tosemopenssl

import itertools
import os
import random
import json
from json import JSONEncoder

def _default(self, obj):
    return getattr(obj.__class__, "to_json", _default.default)(obj)

_default.default = JSONEncoder().default
JSONEncoder.default = _default

from pathos import multiprocessing
import logging
import copy
from pathlib import Path
from typing import Tuple
import re
from z3.z3 import Solver, And, Or, Not, Bool, Int, sat

inputLocation = f"experimentResults/{tool}{program}.json"
outputLocation = f"preparedExperimentResults/{tool}{program}.json"

base: Path = Path(inputLocation)

with open(base) as f:
    base = json.load(f)

def EQ(a,b):
    if a['original_line'] != 'ERROR':
        if 'verified' in a.keys() and 'verified' in b.keys():
            return a['sanitized_message'] == b['sanitized_message'] and a['input_file'] == b['input_file'] and a['original_line'] == b['original_line'] and a['feasible'] == b['feasible'] and a['verified'] == b['verified']
        else:
            return a['sanitized_message'] == b['sanitized_message'] and a['input_file'] == b['input_file'] and a['original_line'] == b['original_line'] and a['feasible'] == b['feasible']
    a1 = a['function_line_range'].split(':')[-1]
    a2 = a['function_line_range'].split(':')[-1]
    b1 = b['function_line_range'].split(':')[-2]
    b2 = b['function_line_range'].split(':')[-1]
    return a1 == b1 and a2 == b2 and a['sanitized_message'] == b['sanitized_message'] and a['input_file'] == b['input_file'] and a['feasible'] == b['feasible']

print(len(base))
i = 0
while i < len(base):
    if not base[i]['feasible']:
        base.pop(i)
        continue
    j = i+1
    while j < len(base):
        if EQ(base[i],base[j]):
            base[i]['presence_condition'] = 'Or(' + base[i]['presence_condition'] + ',' + base[j]['presence_condition'] +')'
            base.pop(j)
        j += 1
    i += 1
print(i)
with open(outputLocation,'w') as x:
    x.write(json.dumps(base,indent=3))

# Format ToSEM Baselines

In [ ]:
program = "busybox" #uclibc, busybox, openssl

import itertools
import os
import random
import json
import re
from pathlib import Path
from json import JSONEncoder

def _default(self, obj):
    return getattr(obj.__class__, "to_json", _default.default)(obj)

inputLocation = f"tosemBaselines/{program}.json"
outputLocation = f"preparedTosemBaselines/{program}.json"

base: Path = Path(inputLocation)

with open(base) as f:
    base = json.load(f)
    
def getType(msg):
    if msg.startswith('Control flow of non-void'): 
        return 'missingReturn'
    elif msg.endswith('freed multiple times!'):
        return 'doubleFree'
    elif msg.endswith('is used uninitialized!'):
        return 'uninitVar'
    elif msg.endswith('is freed although not dynamically allocted!'):
        return 'freeStatic'
    elif msg.endswith('is a dead store!'):
        return 'deadStore'
    elif msg == 'Case statement is not terminated by a break!':
        return 'noBreakSwitch'
    elif msg.startswith('incompatible pointer types'):
        return 'incompatiblePointers'
    elif msg.startswith("assignment discards 'const' qualifier"):
        return 'discardQualifier'
    elif msg == 'warning: switch statement has dangling code':
        return 'danglingSwitch'
    print(msg)
    err = err # bad break
    
newRes = []
for b in base:
    r = {}
    f = re.search(r'TypeChef-Sampling-(OpenSSL|Busybox|uClibc)([a-zA-Z0-9\-\_/\.]+\.c):(\d+):',b['err'])
    if f is None:
        if '.h' in b['err']:
            continue
        print(b['err'])
        err = err # bad break
    r['input_file'] = f.group(2)
    r['input_line'] = int(f.group(3))
    f = re.search('W \[(.*)\]',b['err'])
    r['condition'] = f.group(1)
    r['message'] = b['msg']
    r['type'] = getType(b['msg'])
    newRes.append(r)
    
with open(outputLocation,'w') as x:
    x.write(json.dumps(newRes,indent=3))

# Compare ToSEM

In [ ]:
program = "uclibc" #uclibc, busybox, openssl

import itertools
import os
import random
import json
import re
from pathlib import Path
from json import JSONEncoder

def _default(self, obj):
    return getattr(obj.__class__, "to_json", _default.default)(obj)

inputLocation = f"preparedTosemBaselines/{program}.json"
cexperimentLocation = f"preparedExperimentResults/clangtosem{program}.json"
pexperimentLocation = f"preparedExperimentResults/phasartosem{program}.json"

base: Path = Path(inputLocation)
clangexpr: Path = Path(cexperimentLocation)
phasarexpr: Path = Path(pexperimentLocation)

expr = []
with open(base) as f:
    base = json.load(f)
with open(clangexpr) as f:
    expr.extend(json.load(f))
with open(phasarexpr) as f:
    expr.extend(json.load(f))
    
def sameFile(base, expr):
    bf = base['input_file']
    ef = expr['input_file']
    ef = ef.replace('/targets','').replace('.desugared','')
    bf = bf.replace('/study/farce-uclibc','')
    return bf == ef
def sameLine(base, expr):
    bl = base['input_line']
    ef = expr['original_line']
    if ef == 'ERROR':
        return False
    start = int(ef.split(':')[0])
    end = int(ef.split(':')[1])
    return bl >= start and bl <= end
def trackExpr(expr,map):
    emsg = expr['message']
    exprtype = 'other'
    if 'Attempt to free released memory' in emsg:
        exprtype = 'doubleFree'
    elif 'is never read' in emsg:
        exprtype = 'deadStore'
    elif 'garbage value' in emsg or 'uninitialized value' in emsg or 'Uninitialized Variable' in emsg:
        exprtype = 'uninitVar'
    map[exprtype] += 1
 
def sameType(base, expr):
    emsg = expr['message']
    exprtype = ''
    if 'Attempt to free released memory' in emsg:
        exprtype = 'doubleFree'
    elif 'is never read' in emsg:
        exprtype = 'deadStore'
    elif 'garbage value' in emsg or 'uninitialized value' in emsg or 'Uninitialized Variable' in emsg:
        exprtype = 'uninitVar'
    return base['type'] == exprtype
    

def match(base, expr):
    return sameFile(base,expr) and sameLine(base,expr) and sameType(base,expr)
    
canMatch = ['doubleFree','uninitVar','deadStore','discardQualifier','freeStatic','incompatiblePointers']
count = 0
matches = 0
matchTypes = {'doubleFree':0,'uninitVar':0,'deadStore':0,'discardQualifier':0,'freeStatic':0,'incompatiblePointers':0}
tosemTypes = {'doubleFree':0,'uninitVar':0,'deadStore':0,'discardQualifier':0,'freeStatic':0,'incompatiblePointers':0, 'other':0}
sugarlyzerTypes = {'doubleFree':0,'uninitVar':0,'deadStore':0,'discardQualifier':0,'freeStatic':0,'incompatiblePointers':0, 'other':0}

for e in expr:
    trackExpr(e,sugarlyzerTypes)

for b in base:
    if b['type'] in canMatch:
        count += 1
        tosemTypes[b['type']] += 1
    else:
        tosemTypes['other'] += 1
    for e in expr:
        if match(b,e):
            matches += 1
            matchTypes[b['type']] += 1 
            break

print('viable alarms',count, 'matched alarms', matches)
print (matchTypes)
print(tosemTypes)
print(sugarlyzerTypes)

    

In [ ]:
import itertools
import os
import random
import json
import re
from pathlib import Path
from json import JSONEncoder

infeasible = 0

for tool in ['clang','infer','phasar']:
    for prog in ['axtls','toybox','busybox','varbugs','tosemuclibc','tosemopenssl','tosembusybox']:
        with open(f'experimentResults/{tool}{prog}.json') as i:
            alarms = json.load(i)
        for a in alarms:
            if not a['feasible']:
                infeasible += 1
print(infeasible)